## 25. Simulation Methods

In this we will see that by generating data in a clever way, we can solve a number of problems such as integrating or maximizing a complicated function.  For integration, we will study 3 methods:

- basic Monte Carlo integration
- importance sampling
- Markov chain Monte Carlo (MCMC)

### 25.1 Bayesian Inference Revisited

Simulation methods are specially useful in Bayesian inference so let us briefly review the main ideas.  Given a prior $f(\theta)$ and data $X^n = (X_1, \dots, X_n)$ the posterior density is

$$ f(\theta | X^n) = \frac{\mathcal{L}(\theta) f(\theta)}{ \int \mathcal{L}(u) f(u) \; du} $$

where $\mathcal{L}(\theta)$ is the likelihood function.  The posterior mean is

$$ \overline{\theta} = \int \theta f(\theta | X^n) \; d\theta = \frac{\int \theta \mathcal{L}(\theta) f(\theta) \; d\theta}{\int \mathcal{L}(\theta) f(\theta) \; d\theta} $$

If $\theta = (\theta_1, \dots, \theta_k)$ is multidimensional, then we might be interested in the posterior for one the components, $\theta_1$, say.  This marginal posterior density is

$$ f(\theta_1 | X^n) = \int \int \cdots \int f(\theta_1, \dots, \theta_k | X^n) \; d\theta_2 \cdots d\theta_k$$

which involves high dimensional integration.

You can see that integrals play a big role in Bayesian inference.  When $\theta$ is high dimensional, it may not be feasible to calculate these integrals analytically.  Simulation methods will often be very helpful.

### 25.2 Basic Monte Carlo Integration

Suppose you want to evaluate the integral $I = \int_a^b h(x) dx$ for some function $h$.  If $h$ is an "easy" function like a polynomial or a trigonometric function then we can do the integral in closed form.  In practice, $h$ can be very complicated and there may be no known closed form expression for $I$.  There are many numerical techniques for evaluating $I$ such as Simpson's rule, the trapezoidal rule, Gaussian quadrature and so on.  In some cases these techniques work very well.  But other times they might not work so well.  In particular, it is hard to extend them to higher dimensions.  Monte Carlo integration is another approach to evaluating $I$ which is notable for its simplicity, generality, and scalability.

Let us begin by writing

$$ I = \int_a^b h(x) dx = \int_a^b h(x) (b - a) \frac{1}{b - a} dx = \int_a^b w(x) f(x) dx$$

where $w(x) = h(x)(b - a)$ and $f(x) = 1 / (b - a)$.  Notice that $f$ is the density for a uniform random variable over $(a, b)$.  Hence,

$$ I = \mathbb{E}_f(w(X)) $$

where $X \sim \text{Uniform}(a, b)$.

Suppose we generate $X_1, \dots, X_n \sim \text{Uniform}(a, b)$ where $N$ is large.  By the law of large numbers,

$$ \hat{I} \equiv \frac{1}{N} \sum_{i=1}^N w(X_i) \xrightarrow{\text{P}} \mathbb{E}(w(X)) = I $$

This is the **basic monte carlo integration method**.  We can also compute the standard error of the estimate,

$$ \hat{\text{se}} = \frac{s}{\sqrt{N}} 
\quad \text{where} \quad
s^2 = \frac{\sum_i (Y_i - \hat{I})^2}{N - 1},
\quad Y_i = w(X_i)
$$

A $1 - \alpha$ confidence interval for $I$ is $\hat{I} \pm z_{\alpha / 2} \hat{\text{se}}$.  We can take $N$ as large as we want and hence make the length of the confidence interval very small.

A simple generalization of the basic method is to consider integrals of the form

$$ I = \int h(x) f(x) \; dx $$

where $f(x)$ is a probability density function.  Taking $f$ to be $\text{Uniform}(a, b)$ gives us the special case above.  The only difference is that now we draw $X_1, \dots, X_N \sim f$ and take

$$ \hat{I} \equiv \frac{1}{N} \sum_{i=1}^N h(X_i) $$

as before.